In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv("properties_main.csv")

In [ ]:
df.shape

(50, 17)

In [ ]:
df = df[["price","bedrooms","bathrooms","sqft_living","sqft_lot","city","statezip","sqft_basement","sqft_above","view","floors","condition","waterfront","yr_built","yr_renovated"]]

In [ ]:
df.head()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,city,statezip,sqft_basement,sqft_above,view,floors,condition,waterfront,yr_built,yr_renovated
0,500000,2,2.0,1700,6000,Snoqualmie Pass,WA 98119,500,1200,1,1.0,3,0,1972,2017
1,610000,4,2.5,2400,7000,Pacific,WA 98119,400,2000,4,1.0,4,0,1998,2012
2,740000,3,2.0,1600,5500,Beaux Arts Village,WA 98119,400,1200,2,1.0,3,0,1982,2018
3,530000,4,3.0,2200,7000,Covington,WA 98133,400,1800,3,1.5,5,0,1995,0
4,620000,3,2.0,1600,5500,Mercer Island,WA 98119,400,1200,1,1.0,3,0,1982,2018


In [ ]:
df.isnull().sum()

price            0
bedrooms         0
bathrooms        0
sqft_living      0
sqft_lot         0
city             0
statezip         0
sqft_basement    0
sqft_above       0
view             0
floors           0
condition        0
waterfront       0
yr_built         0
yr_renovated     0
dtype: int64

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

class MultiColumnLabelEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, columns=None):
        self.columns = columns

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        for col in self.columns:
            le = LabelEncoder()
            X[col] = le.fit_transform(X[col])
        return X

X = df.drop(["price"], axis=1)
y = df["price"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

label_encode_cols = ['city', 'statezip']
scale_cols = [col for col in X.columns if col not in label_encode_cols]

preprocessor = ColumnTransformer(
    transformers=[
        ('label_encode', MultiColumnLabelEncoder(columns=label_encode_cols), label_encode_cols),
        ('scale', StandardScaler(), scale_cols)
    ],
    remainder='passthrough'
)

pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor())
])

pipeline.fit(X_train, y_train)

y_pred = pipeline.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')


Mean Squared Error: 2411066250.0


In [ ]:
from sklearn.metrics import r2_score
r2 = r2_score(y_test, y_pred)
print('R-squared Score:', r2)

R-squared Score: 0.648583843463052
